# 实验三：三因子模型的实证检验

## 数据准备与导入

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import f

In [2]:
data_factors = pd.read_csv(
    '../../实验三/assets/data/course/Data_FFFactors.csv',
    encoding='gbk',
    usecols=[2, 6, 7, 8]
)
# 市场因子(mkt)、规模因子(smb)和账面市值比因子(hml)
data_factors.columns = ['date', 'mkt', 'smb', 'hml']
data_factors

,date,mkt,smb,hml
0,2009/1/23,0.0934,0.0516,-0.0265
1,2009/2/27,0.0461,0.0417,0.0116
2,2009/3/31,0.1397,0.0565,-0.0355
3,2009/4/30,0.0441,0.0071,0.0293
4,2009/5/27,0.0626,0.0429,-0.0040
...,...,...,...,...
103,2017/8/31,0.0246,0.0078,-0.0146
104,2017/9/29,-0.0062,-0.0116,-0.0295
105,2017/10/31,0.0110,-0.0609,-0.0310
106,2017/11/30,-0.0259,-0.0371,0.0280


In [3]:
# 重命名date
data_factors['date'] = pd.to_datetime(data_factors['date'])
data_factors['yearmonth'] = data_factors['date'].dt.strftime('%Y%m')

data_factors

,date,mkt,smb,hml,yearmonth
0,2009-01-23,0.0934,0.0516,-0.0265,200901
1,2009-02-27,0.0461,0.0417,0.0116,200902
2,2009-03-31,0.1397,0.0565,-0.0355,200903
3,2009-04-30,0.0441,0.0071,0.0293,200904
4,2009-05-27,0.0626,0.0429,-0.0040,200905
...,...,...,...,...,...
103,2017-08-31,0.0246,0.0078,-0.0146,201708
104,2017-09-29,-0.0062,-0.0116,-0.0295,201709
105,2017-10-31,0.0110,-0.0609,-0.0310,201710
106,2017-11-30,-0.0259,-0.0371,0.0280,201711


In [4]:
data_index = pd.read_csv('../../实验三/assets/data/course/Data_Index.csv', encoding='gbk', usecols=[1, 2, 4])
data_index.columns = ['idxname', 'date', 'return']
data_index['date'] = pd.to_datetime(data_index['date'])
data_index['yearmonth'] = data_index['date'].dt.strftime('%Y%m')
data_index

,idxname,date,return,yearmonth
0,上证指数,1997-01-31,0.0520,199701
1,上证指数,1997-02-28,0.0783,199702
2,上证指数,1997-03-31,0.1868,199703
3,上证指数,1997-04-30,0.1289,199704
4,上证指数,1997-05-30,-0.0779,199705
...,...,...,...,...
1327,上证公用,2017-08-31,-0.0096,201708
1328,上证公用,2017-09-29,-0.0234,201709
1329,上证公用,2017-10-31,0.0056,201710
1330,上证公用,2017-11-30,-0.0238,201711


In [5]:
# 查看一下一共有多少指数
idx_names = np.unique(data_index['idxname'].values)
idx_names

array(['上证信息', '上证公用', '上证医药', '上证可选', '上证工业', '上证指数', '上证材料', '上证消费',
       '上证电信', '上证能源', '上证金融'], dtype=object)

In [6]:
# 提取出不同的指数
data_xinxi = data_index[data_index['idxname'] == idx_names[0]]
data_gongyong = data_index[data_index['idxname'] == idx_names[1]]
data_yiyao = data_index[data_index['idxname'] == idx_names[2]]
data_kexuan = data_index[data_index['idxname'] == idx_names[3]]
data_gongye = data_index[data_index['idxname'] == idx_names[4]]
data_cailiao = data_index[data_index['idxname'] == idx_names[6]]
data_xiaofei = data_index[data_index['idxname'] == idx_names[7]]
data_dianxin = data_index[data_index['idxname'] == idx_names[8]]
data_nengyuan = data_index[data_index['idxname'] == idx_names[9]]
data_jinrong = data_index[data_index['idxname'] == idx_names[10]]

data_dianxin

,idxname,date,return,yearmonth
1116,上证电信,2009-01-23,NaN,200901
1117,上证电信,2009-02-27,0.0331,200902
1118,上证电信,2009-03-31,0.1568,200903
1119,上证电信,2009-04-30,0.1561,200904
1120,上证电信,2009-05-27,-0.0582,200905
...,...,...,...,...
1219,上证电信,2017-08-31,0.0440,201708
1220,上证电信,2017-09-29,0.1165,201709
1221,上证电信,2017-10-31,-0.0105,201710
1222,上证电信,2017-11-30,0.0152,201711


In [7]:
data_rf = pd.read_csv('../../实验三/assets/data/course/Data_RiskFreeReturn.csv', encoding='gbk', usecols=[0, 3])
data_rf.columns = ['date', 'rfreturn']
data_rf['date'] = pd.to_datetime(data_rf['date'])
data_rf['yearmonth'] = data_rf['date'].dt.strftime('%Y%m')
data_rf

,date,rfreturn,yearmonth
0,2009-01-01,0.001256,200901
1,2009-02-01,0.001088,200902
2,2009-03-01,0.001041,200903
3,2009-04-01,0.001013,200904
4,2009-05-01,0.001010,200905
...,...,...,...
103,2017-08-01,0.003602,201708
104,2017-09-01,0.003650,201709
105,2017-10-01,0.003641,201710
106,2017-11-01,0.003798,201711


In [8]:
# 合并所有的pd Dataframe
data_matrix = pd.merge(
    left=data_factors[['yearmonth', 'date', 'mkt', 'smb', 'hml']],
    right=data_xinxi[['yearmonth', 'return']],
    on=['yearmonth'],
    how='inner'
)

data_matrix = pd.merge(
    left=data_matrix,
    right=data_gongyong[['yearmonth', 'return']],
    on=['yearmonth'],
    how='inner',
    suffixes=['', '_gy']  # 加上后缀，收益率后缀
)

data_matrix = pd.merge(
    left=data_matrix,
    right=data_yiyao[['yearmonth', 'return']],
    on=['yearmonth'],
    how='inner',
    suffixes=['', '_yy']
)

data_matrix = pd.merge(
    left=data_matrix,
    right=data_kexuan[['yearmonth', 'return']],
    on=['yearmonth'],
    how='inner',
    suffixes=['', '_kx']
)

data_matrix = pd.merge(
    left=data_matrix,
    right=data_gongye[['yearmonth', 'return']],
    on=['yearmonth'],
    how='inner',
    suffixes=['', '_ggy']
)

data_matrix = pd.merge(
    left=data_matrix,
    right=data_cailiao[['yearmonth', 'return']],
    on=['yearmonth'],
    how='inner',
    suffixes=['', '_cl']
)

data_matrix = pd.merge(
    left=data_matrix,
    right=data_xiaofei[['yearmonth', 'return']],
    on=['yearmonth'],
    how='inner',
    suffixes=['', '_xf']
)

data_matrix = pd.merge(
    left=data_matrix,
    right=data_dianxin[['yearmonth', 'return']],
    on=['yearmonth'],
    how='inner',
    suffixes=['', '_dx']
)

data_matrix = pd.merge(
    left=data_matrix,
    right=data_nengyuan[['yearmonth', 'return']],
    on=['yearmonth'],
    how='inner',
    suffixes=['', '_ny']
)

data_matrix = pd.merge(
    left=data_matrix,
    right=data_jinrong[['yearmonth', 'return']],
    on=['yearmonth'],
    how='inner',
    suffixes=['', '_jr']
)

data_matrix = pd.merge(
    left=data_matrix,
    right=data_rf[['yearmonth', 'rfreturn']],
    on=['yearmonth'],
    how='inner'
)

data_matrix

,yearmonth,date,mkt,smb,hml,return,return_gy,return_yy,return_kx,return_ggy,return_cl,return_xf,return_dx,return_ny,return_jr,rfreturn
0,200901,2009-01-23,0.0934,0.0516,-0.0265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001256
1,200902,2009-02-27,0.0461,0.0417,0.0116,0.0560,0.0609,0.0379,0.0954,0.0330,0.0947,0.1007,0.0331,-0.0270,0.0347,0.001088
2,200903,2009-03-31,0.1397,0.0565,-0.0355,0.1915,0.1698,0.1050,0.1934,0.1909,0.2219,0.0892,0.1568,0.1794,0.1739,0.001041
3,200904,2009-04-30,0.0441,0.0071,0.0293,0.0440,0.0132,0.0639,0.1022,0.0329,0.0102,0.0642,0.1561,0.1426,0.0281,0.001013
4,200905,2009-05-27,0.0626,0.0429,-0.0040,0.0365,0.0475,-0.0395,0.0552,0.0207,0.0456,-0.0026,-0.0582,0.1239,0.0731,0.001010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,201708,2017-08-31,0.0246,0.0078,-0.0146,0.0562,-0.0096,0.0172,-0.0027,-0.0030,0.0432,0.0310,0.0440,0.0097,0.0333,0.003602
104,201709,2017-09-29,-0.0062,-0.0116,-0.0295,0.0490,-0.0234,0.0249,0.0307,-0.0219,-0.0042,0.0816,0.1165,-0.0055,-0.0230,0.003650
105,201710,2017-10-31,0.0110,-0.0609,-0.0310,0.0017,0.0056,0.0830,0.0258,0.0392,-0.0340,0.0789,-0.0105,-0.0357,0.0329,0.003641
106,201711,2017-11-30,-0.0259,-0.0371,0.0280,-0.0257,-0.0238,-0.0380,-0.0452,-0.0228,0.0178,-0.0435,0.0152,0.0209,0.0355,0.003798


In [9]:
data_matrix.columns = ['yearmonth', 'date', 'mkt', 'smb', 'hml', 'xinxi', 'gongyong', 'yiyao', 'kexuan',
                       'gongye', 'cailiao', 'xiaofei', 'dianxin', 'nengyuan', 'jinrong', 'rfreturn']
data_matrix.dropna(inplace=True)
data_matrix.sort_values(by='date', inplace=True)
data_matrix

,yearmonth,date,mkt,smb,hml,xinxi,gongyong,yiyao,kexuan,gongye,cailiao,xiaofei,dianxin,nengyuan,jinrong,rfreturn
1,200902,2009-02-27,0.0461,0.0417,0.0116,0.0560,0.0609,0.0379,0.0954,0.0330,0.0947,0.1007,0.0331,-0.0270,0.0347,0.001088
2,200903,2009-03-31,0.1397,0.0565,-0.0355,0.1915,0.1698,0.1050,0.1934,0.1909,0.2219,0.0892,0.1568,0.1794,0.1739,0.001041
3,200904,2009-04-30,0.0441,0.0071,0.0293,0.0440,0.0132,0.0639,0.1022,0.0329,0.0102,0.0642,0.1561,0.1426,0.0281,0.001013
4,200905,2009-05-27,0.0626,0.0429,-0.0040,0.0365,0.0475,-0.0395,0.0552,0.0207,0.0456,-0.0026,-0.0582,0.1239,0.0731,0.001010
5,200906,2009-06-30,0.1331,-0.0208,-0.0079,0.0041,0.0220,0.0775,0.0434,0.0612,0.1364,0.0918,0.0841,0.0935,0.2472,0.001043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,201708,2017-08-31,0.0246,0.0078,-0.0146,0.0562,-0.0096,0.0172,-0.0027,-0.0030,0.0432,0.0310,0.0440,0.0097,0.0333,0.003602
104,201709,2017-09-29,-0.0062,-0.0116,-0.0295,0.0490,-0.0234,0.0249,0.0307,-0.0219,-0.0042,0.0816,0.1165,-0.0055,-0.0230,0.003650
105,201710,2017-10-31,0.0110,-0.0609,-0.0310,0.0017,0.0056,0.0830,0.0258,0.0392,-0.0340,0.0789,-0.0105,-0.0357,0.0329,0.003641
106,201711,2017-11-30,-0.0259,-0.0371,0.0280,-0.0257,-0.0238,-0.0380,-0.0452,-0.0228,0.0178,-0.0435,0.0152,0.0209,0.0355,0.003798


## 模型构建与检验

三因子回归：

Ri ​− Rf ​= αi ​+ βi​(Rm ​− Rf​) + si​SMB + hi​HML + εi​

### 单资产检验

In [10]:
# []取行 [[]]取列
x = data_matrix[['mkt', 'smb', 'hml']].values
rf = data_matrix[['rfreturn']].values
rstock = data_matrix[['nengyuan']].values

In [11]:
X = sm.add_constant(x) # 加上常数α
Y = rstock - rf # 超额收益率
model = sm.OLS(Y, X)
res = model.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.793
Model:                            OLS   Adj. R-squared:                  0.787
Method:                 Least Squares   F-statistic:                     131.4
Date:                Tue, 15 Apr 2025   Prob (F-statistic):           4.55e-35
Time:                        19:40:49   Log-Likelihood:                 191.69
No. Observations:                 107   AIC:                            -375.4
Df Residuals:                     103   BIC:                            -364.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0068      0.004     -1.637      0.1

F统计量为131.4，极其显著，表明模型整体有效

### 多资产检验

T为样本数量，N为资产数量(10个行业)，K为因子数量(3个)。y是10个行业的超额收益率矩阵。

In [12]:
T = len(Y)
N = 10
K = 3

# iloc用数字定位，loc用名字定位
y = data_matrix.iloc[:, 5:15].values - data_matrix.loc[:, ['rfreturn']].values
x = data_matrix.loc[:, ['mkt', 'smb', 'hml']].values
x = sm.add_constant(x)

In [13]:
# 手动计算最小二乘法
xTx = np.dot(np.transpose(x), x)
xTy = np.dot(np.transpose(x), y)
AB_hat = np.dot(
    np.linalg.inv(xTx), xTy
)
# AB_hat 分别为，α，β1， β2，β3
AB_hat

array([[ 2.20998281e-03, -4.75559083e-03,  8.69213766e-03,
         6.01200503e-04, -2.48682375e-03, -1.86367251e-03,
         6.65327137e-03,  1.08445833e-03, -6.83546927e-03,
         3.81171884e-03],
       [ 1.09169613e+00,  8.82775236e-01,  6.92090324e-01,
         9.68395538e-01,  1.17452390e+00,  1.19894349e+00,
         7.79437498e-01,  1.06528553e+00,  1.07107511e+00,
         1.10645790e+00],
       [ 3.18002300e-01,  1.56667695e-01,  5.46265805e-02,
         2.23503381e-01, -5.26822562e-02,  4.32004979e-02,
         6.03631437e-02,  2.73720526e-01, -3.69911640e-02,
        -2.95489243e-01],
       [-1.01018531e+00,  3.11744611e-01, -9.49843743e-01,
        -4.03680938e-01,  1.44676804e-01, -1.36339861e-01,
        -5.46376034e-01, -8.59904872e-01,  2.41287277e-01,
         3.74178168e-01]])

In [14]:
ALPHA = AB_hat[0]
ALPHA

array([ 0.00220998, -0.00475559,  0.00869214,  0.0006012 , -0.00248682,
       -0.00186367,  0.00665327,  0.00108446, -0.00683547,  0.00381172])

In [15]:
# 真实 - 预测
RESD = y - np.dot(x, AB_hat)
RESD

array([[ 0.00083228,  0.01372237,  0.00495476, ..., -0.01962137,
        -0.07188549, -0.01322599],
       [-0.01808964,  0.0524061 , -0.03822401, ..., -0.04013768,
         0.04622098,  0.04445358],
       [ 0.01997383, -0.03223425,  0.05111625, ...,  0.13027525,
         0.09438098, -0.03438496],
       ...,
       [-0.02810904,  0.01620921,  0.03693547, ..., -0.03693107,
        -0.03906021,  0.00688047],
       [ 0.03665002, -0.00289501, -0.00394273, ...,  0.0721408 ,
         0.0435499 ,  0.0351079 ],
       [-0.0211494 , -0.00546887,  0.01830547, ..., -0.04703806,
         0.01312874, -0.02137116]])

In [16]:
COV = np.dot(
    np.transpose(RESD), RESD
) / T
invCOV = np.linalg.inv(COV)

In [17]:
# 三因子数据，不含常数α
fs = x[:, [1, 2, 3]]

# 计算三因子的均值
# 沿着行方向（axis=0）计算平均值
# 沿着行方向即每列求均值，可以理解为从下往上压缩，最后得到一个 1x3的矩阵
muhat = np.mean(fs, axis=0).reshape((3, 1))
muhat

array([[0.00620654],
       [0.00946262],
       [0.00135607]])

In [18]:
# 标准化，x -> (x - μ)
fs = fs - np.mean(fs, axis=0)

In [ ]:
# 协方差矩阵
omegahat = np.dot(np.transpose(fs), fs)/T
invOMG = np.linalg.inv(omegahat)

# 对角线元素是各因子的方差，非对角线元素是因子之间的协方差
omegahat

array([[ 0.0051427 ,  0.00034092,  0.00046346],
       [ 0.00034092,  0.00160373, -0.00095725],
       [ 0.00046346, -0.00095725,  0.00161024]])

$$
GRS = \frac{T-N-K}{N} \times \frac{1}{1+\mu_f^T \Omega^{-1} \mu_f} \times \alpha^T \Sigma^{-1} \alpha
$$

In [21]:
# μ_f' × Ω^(-1) × μ_f
xxx = np.dot(
    np.dot(
        np.transpose(muhat), invOMG
    ), muhat
)

# α' × Σ^(-1) × α
yyy = np.dot(
    np.dot(
        ALPHA, invCOV
    ), np.transpose(ALPHA)
)

GRS = (T - N - K) / N * (1/ (1 + xxx)) * yyy
GRS

array([[1.41821139]])

GRS统计量服从自由度为(N, T-N-K)的F分布

In [22]:
# f.cdf(GRS[0][0], N, T-N-K)计算GRS值对应的F分布累积概率
# 1 - f.cdf(...)得到右尾概率，即p值
pvalue = 1 - f.cdf(GRS[0][0], N, T - N - K)
pvalue

0.18428215616746135

GRS检验的原假设是：所有资产的alpha值同时为零（H0: α1 = α2 = ... = αN = 0）。

- 如果p值小于显著性水平（如0.05），则拒绝原假设，说明至少有一个α显著不为零，模型存在定价错误

- 如果p值大于显著性水平，则不能拒绝原假设，说明模型在统计上是有效的

在这个实验中，GRS=1.4182，p值=0.1843 > 0.05，说明我们不能拒绝"所有alpha同时为零"的原假设，意味着三因子模型能够很好地解释这10个行业投资组合的收益率，不存在显著的定价异常。

In [ ]:
print('三因子模型的多资产检验结果')
# :>7s表示右对齐、宽度为7个字符的字符串格式
print('{:>7s},{:>7s},{:>7s},{:>7s},{:>7s},{:>7s},{:>7s},{:>7s},{:>7s}'.format('alpha1', 'alpha2', 'alpha3', 'alpha4', 'alpha5', 'alpha6', 'alpha7', 'GRS', 'pvalue'))
# {:7.4f} 是格式化浮点数的模板，表示总宽度为7个字符，小数点后保留4位的浮点数格式
print('{:7.4f},{:7.4f},{:7.4f},{:7.4f},{:7.4f},{:7.4f},{:7.4f},{:7.4f},{:7.4f}'.format(ALPHA[0], ALPHA[1], ALPHA[2], ALPHA[3], ALPHA[4], ALPHA[5], ALPHA[6], GRS[0][0], pvalue))

三因子模型的多资产检验结果
 alpha1, alpha2, alpha3, alpha4, alpha5, alpha6, alpha7,    GRS, pvalue
 0.0022,-0.0048, 0.0087, 0.0006,-0.0025,-0.0019, 0.0067, 1.4182, 0.1843
